In [1]:
from collections import defaultdict
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager 

# Programs
Retrieve card names and generate input of URLs to scrape

In [2]:
def retrieve_name_url(card_table):
    """
    take in a table from the website https://shop.tcgplayer.com/price-guide/final-fantasy-tcg/
    with the class attribute "priceGuideTable tablesorter"
    and outputs a DataFrame of the names of cards in the table and a link to their description
    """
    card_names = []
    url_inputs = []

    for row in card_table.tbody.find_all('tr'):
        name_div = row.td.find('div',class_='productDetail')
        card_name = name_div.get_text().strip('\n')
        url_input = name_div.a.attrs['href']

        card_names.append(card_name)
        url_inputs.append(url_input)

    url_df = pd.DataFrame()
    url_df['name'] = pd.Series(card_names)
    url_df['url_inputs'] = pd.Series(url_inputs)
    return url_df

Sub function to retrieve prices from price table in the URL of the trading card

In [3]:
def retrieve_prices(soup,row_data):
    """
    takes in a Beautiful Soup from a TCGPlayer card page
    and an existing dictionary of row data
    then identifies the table of prices
    and updates the dictionary with the price information included
    """
    
    mkt_price_table = soup.find('div',class_="price-point price-point--market").table
    mid_price_table = soup.find('div',class_="price-point price-point--listed-median").table

    for i in range(len(mkt_price_table.find_all('td'))):
        data = mkt_price_table.find_all('td')
        headers = mkt_price_table.find_all('th')[1:]
        var_name = "TCG" + headers[i].get_text()
        row_data[var_name] = data[i].get_text().strip("$")

    for i in range(len(mid_price_table.find_all('td'))):
        data = mid_price_table.find_all('td')
        headers = mid_price_table.find_all('th')[1:]
        var_name = "Median" + headers[i].get_text()
        row_data[var_name] = data[i].get_text().strip("$")
    
    return row_data


Scrape Card Data from each URL

In [4]:
def scrape_card_data(url_df):
    """
    Takes in a DataFrame of card names and their associated URLs for a FF TCG set
    and returns a DataFrame of the resulting card's attributes for each card 
    """
    data_dict = defaultdict(dict)
    for i, row in url_df.iterrows():
        url = row['url_inputs']
        name = row['name']
        response = requests.get(url)
        if response.status_code == 200:
            page = response.text
            soup = BeautifulSoup(page)
            info_table = soup.find('table',class_='table')

            list_of_dts = info_table.find_all('dt')
            list_of_dds = info_table.find_all('dd')

            row_data = defaultdict(str)
            
            for i in range(len(list_of_dts)):
                var = list_of_dts[i].get_text().strip(":")
                value = list_of_dds[i].get_text()
                row_data[var] = value
                
            row_data['URL'] = url
            
            row_data = retrieve_prices(soup,row_data)
    
            data_dict[name] = row_data
    card_df = pd.DataFrame.from_dict(data_dict,orient='index').reset_index()
    card_df = card_df.rename(columns={'index':'Name'})
    card_df['id'] = card_df.index
    card_df = pd.wide_to_long(card_df,stubnames=['TCG','Median'],i='id',j='Card Material',suffix="\\S+")
    card_df = card_df.reset_index(level='Card Material')

    return card_df.sort_values(by=['Name','Card Material'],ascending=[True,False]).reset_index(drop=True)

# Main Webscraping Code

#### Set up web driver

In [5]:
chromedriver = "C:\\Users\\wlee\\.wdm\\drivers\\chromedriver\\87.0.4280.88\\win32\\chromedriver.exe" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

#### Open browser and retrieve HTML

In [6]:
ref_browser =webdriver.Chrome(chromedriver)

url = "https://shop.tcgplayer.com/price-guide/final-fantasy-tcg"
ref_browser.get(url)

ref_HTML = ref_browser.execute_script("return document.body.innerHTML")
ref_soup = BeautifulSoup(ref_HTML,'html.parser')

#### Retrieve available card sets for FF TCG from drop-down menu

In [7]:
set_names = []

for value in ref_soup.find("select",id="set").find_all('option'):
    set_names.append(value.get_text())
print(set_names[:5])

['Opus XII', 'Opus XI', 'Opus X', 'Opus IX', 'Opus VIII']


#### Let the scrape begin!
Selenium will loop through each set from the Set drop down menu of the website and begin scraping TCGPlayer.com information for each of the cards for each set. The resulting DataFrame will be named _all_cards_.

In [8]:
all_cards = pd.DataFrame()

for set_name in set_names:
    select = Select(ref_browser.find_element_by_id("set"))
    select.select_by_visible_text(set_name)
    
    ref_HTML = ref_browser.execute_script("return document.body.innerHTML")
    ref_soup = BeautifulSoup(ref_HTML,'html.parser')
    
    card_table = ref_soup.find('table',class_='priceGuideTable tablesorter')
    url_df = retrieve_name_url(card_table)
    card_df = scrape_card_data(url_df)
    card_df['set'] = set_name
    all_cards = all_cards.append(card_df, sort = True)

all_cards = all_cards.reset_index().drop(columns='index')

In [9]:
all_cards['Element'].value_counts()

Fire               642
Earth              634
Wind               633
Lightning          621
Ice                606
Water              604
Light               76
Dark                66
Earth;Lightning      8
Fire;Ice             8
Earth;Wind           6
Fire;Water           6
Lightning;Water      6
Ice;Wind             4
Water;Wind           2
Name: Element, dtype: int64

In [ ]:
all_cards.sample(n=5)

In [12]:
all_cards.to_csv("card_information.csv", index=False)

In [13]:
ref_browser.close()

## Program Testing

In [179]:
test_browser =webdriver.Chrome(chromedriver)

url = "https://shop.tcgplayer.com/price-guide/final-fantasy-tcg/deck-exclusive-cards"
test_browser.get(url)

test_HTML = test_browser.execute_script("return document.body.innerHTML")
test_soup = BeautifulSoup(test_HTML,'html.parser')

card_table = test_soup.find('table', class_="priceGuideTable tablesorter")

# retrieve_name_url called here
url_df = retrieve_name_url(card_table)
test_browser.close()
url_df.head()

,name,url_inputs
0,Aerith (Common) - 1-065C,https://shop.tcgplayer.com/final-fantasy-tcg/d...
1,Amarant - 3-013R,https://shop.tcgplayer.com/final-fantasy-tcg/d...
2,Amon - 2-098L,https://shop.tcgplayer.com/final-fantasy-tcg/d...
3,Archer (Male Unit) - 1-088C,https://shop.tcgplayer.com/final-fantasy-tcg/d...
4,Arecia Al-Rashia - 3-097R,https://shop.tcgplayer.com/final-fantasy-tcg/d...


In [180]:
# scrape_card_data called here (test only small number of rows)
card_df = scrape_card_data(url_df.iloc[-7:])

# card_df = card_df.rename(columns={'TCGNormal':'TCG0','TCGFoil':'TCG1','MedianNormal':'Median0','MedianFoil':'Median1'})
card_df.head(10)



,Card Material,Element,Job,Cost,Name,Number,Description,Card Type,Rarity,Category,URL,Power,TCG,Median
0,Normal,NaN,NaN,NaN,Steiner EX - 3-137R,3-137R,"EX BURST When Steiner enters the field, you ma...",NaN,Rare,NaN,https://shop.tcgplayer.com/final-fantasy-tcg/d...,NaN,0.25,0.25
1,Normal,Ice,Standard Unit,1,Summoner (Female Unit) - 1-040C,1-040C,This card was reprinted with a new formatted s...,Backup,Common,FFT,https://shop.tcgplayer.com/final-fantasy-tcg/d...,NaN,0.65,0.65
2,Normal,Lightning,Standard Unit,1,Summoner (Male Unit) - 1-138C,1-138C,This card was reprinted with a new formatted s...,Backup,Common,FFT,https://shop.tcgplayer.com/final-fantasy-tcg/d...,NaN,0.34,0.42
3,Normal,Water,Viking,2,Viking (Refia) - 1-167C,1-167C,"When Viking enters the field, draw 1 card.\r\n...",Forward,Common,III,https://shop.tcgplayer.com/final-fantasy-tcg/d...,1000,0.65,0.60
4,Normal,Earth,Gunslinger,5,Vincent - 1-094R,1-094R,"When Vincent attacks, Vincent gains First Stri...",Forward,Rare,VII,https://shop.tcgplayer.com/final-fantasy-tcg/d...,9000,0.43,0.43
5,Normal,Fire,Black Mage,3,Vivi - 3-017L,3-017L,"When Vivi enters the field, choose 1 Forward o...",Forward,Legend,IX,https://shop.tcgplayer.com/final-fantasy-tcg/d...,4000,1.15,1.25
6,Normal,Wind,Standard Unit,3,White Mage - 2-058C,2-058C,"When White Mage enters the field, choose up to...",Backup,Common,XIV,https://shop.tcgplayer.com/final-fantasy-tcg/d...,NaN,0.54,0.64


In [177]:
# Test scrape of price

# response = requests.get(url_df['url_inputs'][0])
response = requests.get("https://shop.tcgplayer.com/final-fantasy-tcg/deck-exclusive-cards/steiner-ex-3-137r")
if response.status_code == 200:
    page = response.text
    soup = BeautifulSoup(page)

mkt_price_table = soup.find('div',class_="price-point price-point--market").table
mid_price_table = soup.find('div',class_="price-point price-point--listed-median").table

for i in range(len(mkt_price_table.find_all('td'))):
    data = mkt_price_table.find_all('td')
    headers = mkt_price_table.find_all('th')[1:]
    var_name = "TCG " + headers[i].get_text()
    mydict[var_name] = data[i].get_text().strip("$")

for i in range(len(mid_price_table.find_all('td'))):
    data = mid_price_table.find_all('td')
    headers = mid_price_table.find_all('th')[1:]
    var_name = "Median " + headers[i].get_text()
    mydict[var_name] = data[i].get_text().strip("$")

mydict

defaultdict(str,
            {'Price': '0.34',
             'TCG Normal': '0.25',
             'TCG Foil': '0.50',
             'Median Normal': '0.25',
             'Median Foil': '0.41'})